<a href="https://colab.research.google.com/github/tharushaliyanagama/OralCancerEarlyDetection-DSGP/blob/Image-Prediction/MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
import os
import random
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Paths to your dataset
data_dir = "/content/drive/MyDrive/Proper_dataset"
train_dir = os.path.join(data_dir, "train")  # Replace with your dataset's train folder
val_dir = os.path.join(data_dir, "val")      # Replace with your dataset's validation folder

In [4]:
# Parameters
batch_size = 32
img_height = 224
img_width = 224

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 2603 images belonging to 2 classes.
Found 651 images belonging to 2 classes.


In [5]:
def create_mobilenetv2_model():
    base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [6]:
def generate_classification_report(model, val_data):
    # Get predictions from the validation data
    val_data.reset()  # Reset the generator to avoid skipping images
    y_true = val_data.classes
    y_pred = model.predict(val_data)
    y_pred = np.round(y_pred).astype(int)  # Convert probabilities to 0 or 1

    # Generate and print classification report
    report = classification_report(y_true, y_pred, target_names=val_data.class_indices.keys())
    print(report)

In [7]:
mobilenetv2_model = create_mobilenetv2_model()

# Print model summaries
print("MobileNetV2 Model Summary:")
mobilenetv2_model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
MobileNetV2 Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           1,281 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [8]:
# Train MobileNetV2 model
print("Training MobileNetV2 model...")
mobilenetv2_history = mobilenetv2_model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

Training MobileNetV2 model...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.8211 - loss: 0.4350 

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


82/82 ━━━━━━━━━━━━━━━━━━━━ 1717s 21s/step - accuracy: 0.8217 - loss: 0.4336 - val_accuracy: 0.9217 - val_loss: 0.2164
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.9241 - loss: 0.2074 - val_accuracy: 0.9339 - val_loss: 0.1761
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.9438 - loss: 0.1721 - val_accuracy: 0.9462 - val_loss: 0.1513
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9498 - loss: 0.1387 - val_accuracy: 0.9447 - val_loss: 0.1367
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.9459 - loss: 0.1404 - val_accuracy: 0.9339 - val_loss: 0.1694
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.9556 - loss: 0.1314 - val_accuracy: 0.9585 - val_loss: 0.1329
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 208s 3s/step - accuracy: 0.9582 - loss: 0.1294 - val_accuracy: 0.9662 - val_loss: 0.1215
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.9568 - loss: 0.1197 - val_accuracy: 0.9647 - val_loss: 